In [1]:
from sklearn.model_selection import train_test_split

In [2]:
quotes_base = []

with open('quotes.txt', encoding='utf-8') as f:
        quotes_base = f.read().split('/n')

In [3]:
quotes_base

['Там, где все горбаты,Стройность становится уродством.Оноре Де Бальзак',
 'Слава богу, я с тобой,А с тобою как в раю.Ф. Тютч',
 'Так чувственно молчать лишь ты умеешь...  Владимир Вишневский',
 'Eсли не видишь нeдостатков в челoвеке,значит, ты влюбилcя.Идиот.— Aнтон Пaвлович Ч',
 'Не бойся, что не получится. Бойся, что не попробуешь.',
 'Самые лучшие люди те, благодаря которым ты улыбаешься чаще всего.',
 'Все испытывают страх в тот миг, когда могут изменить свою судьбу.',
 'Общение — это как секс: нужно ещё вызвать желание у человека этим с тобой заниматься. Всё остальное — насилие.',
 'усть любовь начнется, но не с тела, а с души, вы слышите? с души. э.',
 'Мы прощались, как во сне, Я сказала: «Жду». Он, смеясь, ответил мне: «Встретимся в аду».  Анна Ахмат',
 'Ревновать - это мстить себе за ошибки других...  Сергей Довлатов "Иностранка"',
 'Злость — самая бесполезная из эмоций. Разрушает мозг и вредит сердцу.  Стивен Кинг. "Тёмная Башня VI: Песнь Сюзанны"',
 'Перемены происходят лиш

In [4]:
train, test = train_test_split(quotes_base,test_size=0.15) 

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
tokenizer.pad_token = 0

In [9]:
from transformers import TextDataset,DataCollatorForLanguageModeling

In [10]:
train, test = train_test_split(quotes_base, test_size=0.15)

In [11]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        wise_words = tokenizer(self.data[idx])['input_ids']
        return wise_words

In [12]:
train_ds = CustomDataset(train)
test_ds = CustomDataset(test)

In [13]:
for i in train_ds:
    print(i)
    break

[11233, 519, 21494, 548, 1381, 538, 9349, 4490, 4109, 18, 24028, 16, 961, 19411, 530, 16, 4703, 29798, 18, 415, 18, 1140, 686, 526, 280, 458, 45065, 1637, 48951, 2334, 6211, 274, 6]


In [14]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

In [15]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model.config

C:\ProgramData\python\lib\site-packages\transformers\models\auto\modeling_auto.py:843: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [16]:
training_args = TrainingArguments(
    output_dir="./overminds", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    evaluation_strategy='steps',
    eval_steps = 800, # Number of update steps between two evaluations.
    save_steps=700, # after # steps model is saved 
    warmup_steps=70,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
trainer.train()

***** Running training *****
  Num examples = 28133
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7034
  7%|▋         | 501/7034 [02:01<25:58,  4.19it/s]

{'loss': 3.3462, 'learning_rate': 4.6912693854106834e-05, 'epoch': 0.14}


 10%|▉         | 700/7034 [02:50<24:43,  4.27it/s]Saving model checkpoint to ./overminds\checkpoint-700
Configuration saved in ./overminds\checkpoint-700\config.json
Model weights saved in ./overminds\checkpoint-700\pytorch_model.bin
 11%|█▏        | 800/7034 [03:18<30:13,  3.44it/s]***** Running Evaluation *****
  Num examples = 4965
  Batch size = 8

 11%|█▏        | 801/7034 [03:51<17:27:06, 10.08s/it]

{'eval_loss': 2.9188785552978516, 'eval_runtime': 32.7234, 'eval_samples_per_second': 151.726, 'eval_steps_per_second': 18.977, 'epoch': 0.23}


 14%|█▍        | 1001/7034 [04:38<23:03,  4.36it/s]

...

100%|█████████▉| 7000/7034 [31:23<00:07,  4.57it/s]Saving model checkpoint to ./overminds\checkpoint-7000
Configuration saved in ./overminds\checkpoint-7000\config.json


{'loss': 2.0371, 'learning_rate': 2.441125789775991e-07, 'epoch': 1.99}


Model weights saved in ./overminds\checkpoint-7000\pytorch_model.bin
100%|██████████| 7034/7034 [31:34<00:00,  4.86it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 7034/7034 [31:35<00:00,  3.71it/s]

{'train_runtime': 1895.0429, 'train_samples_per_second': 29.691, 'train_steps_per_second': 3.712, 'train_loss': 2.500602602856994, 'epoch': 2.0}


TrainOutput(global_step=7034, training_loss=2.500602602856994, metrics={'train_runtime': 1895.0429, 'train_samples_per_second': 29.691, 'train_steps_per_second': 3.712, 'train_loss': 2.500602602856994, 'epoch': 2.0})

In [19]:
trainer.save_model()

Saving model checkpoint to ./overminds
Configuration saved in ./overminds\config.json
Model weights saved in ./overminds\pytorch_model.bin


In [20]:
from transformers import pipeline

wise_words = pipeline(
    "text-generation",
    model='./overminds',
    tokenizer='sberbank-ai/rugpt3small_based_on_gpt2',   
    )

C:\Users\zald\AppData\Roaming\Python\Python39\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
loading configuration file ./overminds\config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.8.2",


In [27]:
wise_words('Партия сказала')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Партия сказала, что не будет играть в шахматы, если ты не будешь играть в шахматы. '}]

In [28]:
wise_words('Народная мудрость')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Народная мудрость гласит: «Чем беднее душа, тем легче нравственность».  © Лев Николаевич'}]

In [30]:
wise_words('Вышел заяц на крыльцо')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Вышел заяц на крыльцо, а следом за ним и волк.  © Лев Николаевич Толстой.'}]

In [31]:
wise_words('Отец сказал сыну')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Отец сказал сыну: "Если бы ты знал, как редко нас понимают правильно, ты бы никогда не'}]

In [33]:
wise_words('Любовь это')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Любовь это когда ничего не стыдно, ничего не страшно, понимаете? Когда тебя не подведут, не'}]

In [35]:
wise_words('Любовь это буря')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Любовь это буря, в которой сгорает все дотла.  © Уильям Шекспир "Король'}]

In [37]:
wise_words('Бутерброд')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Бутерброд с ветчиной — это не обед, а ужин.  © Оноре де'}]